<a href="https://colab.research.google.com/github/advik-7/Image_Encryption/blob/main/CCGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# --- Combined Generator Function ---
def conditional_cyclic_generator():
    # --- 1. Build the Generator Model ---
    def build_generator():
        image_input = layers.Input(shape=(256, 256, 3))  # Image input
        key_input = layers.Input(shape=(16,))  # Conditional key input (size 16)

        # Process the key and map it to spatial format
        key = layers.Dense(256 * 256)(key_input)
        key = layers.Reshape((256, 256, 1))(key)  # Reshape to match image dimensions

        # Concatenate the image and the key
        combined_input = layers.Concatenate()([image_input, key])

        # Encoder: Downsampling layers
        x = layers.Conv2D(64, kernel_size=4, strides=2, padding='same')(combined_input)
        x = layers.ReLU()(x)
        x = layers.Conv2D(128, kernel_size=4, strides=2, padding='same')(x)
        x = layers.ReLU()(x)

        # Decoder: Upsample back to image dimensions
        x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same')(x)
        x = layers.ReLU()(x)
        x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)
        x = layers.ReLU()(x)

        # Output encrypted image
        output_image = layers.Conv2D(3, kernel_size=7, strides=1, padding='same', activation='tanh')(x)

        return Model([image_input, key_input], output_image)

    # Loss Functions
    mae_loss = tf.keras.losses.MeanAbsoluteError()

    def cycle_consistency_loss(real_image, cycled_image):
        return mae_loss(real_image, cycled_image)

    # Optimize
    optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    # Training Step
    def train_step(generator, real_image, key):
        with tf.GradientTape() as tape:
            # Encrypt the real image
            encrypted_image = generator([real_image, key], training=True)

            # cycle consistency
            decrypted_image = generator([encrypted_image, key], training=True)

            # cycle-consistency loss
            cycle_loss = cycle_consistency_loss(real_image, decrypted_image)

        gradients = tape.gradient(cycle_loss, generator.trainable_variables)
        optimizer.apply_gradients(zip(gradients, generator.trainable_variables))

        return cycle_loss
